# Mixtures of conjugate priors 01

# JAGS treatment

# _Josep Fortiana_ $\hskip6cm$ 2022-04-04

## The spinning coin

***
###### LaTeX macros
$\def\prob{P}$
$\def\argmax{\operatorname{arg\,max}}$
$\def\argmin{\operatorname{arg\,min}}$
$\def\borel{\operatorname{Borel}}$
$\def\cE{\cal E}$
$\def\cP{\cal P}$
$\def\R{\mathbb{R}}$ 
$\def\N{\mathbb{N}}$
$\def\Z{\mathbb{Z}}$
$\def\Ee{\operatorname{E}}$
$\def\va{\text{v.a.}}$
$\def\var{\operatorname{var}}$
$\def\cov{\operatorname{cov}}$
$\def\cor{\operatorname{cor}}$
$\def\binomdist{\operatorname{Binom}}$
$\def\berndist{\operatorname{Ber}}$
$\def\betabinomdist{\operatorname{Beta-Binom}}$
$\def\betadist{\operatorname{Beta}}$
$\def\expdist{\operatorname{Exp}}$
$\def\gammadist{\operatorname{Gamma}}$
$\def\hyperdist{\operatorname{Hypergeom}}$
$\def\hypergeomdist{\operatorname{Hypergeom}}$
$\DeclareMathOperator{\multinomialdist}{Multinomial}$
$\DeclareMathOperator{\multinomdist}{Multinom}$
$\def\poissondist{\operatorname{Poisson}}$
$\def\geomdist{\operatorname{Geom}}$
$\def\normaldist{\operatorname{N}}$
$\def\unifdist{\operatorname{Unif}}$
$\DeclareMathOperator{\indica}{\mathbb{1}}$
$\def\CondTo{\mathbin{|\mskip0.5mu}}$
***

In [ ]:
#install.packages("R2jags",dependencies=TRUE,repos= "https://cloud.r-project.org")
require(R2jags)

## Data 

In [ ]:
prior.alpha1<-10
prior.beta1<-20
prior.alpha2<-20
prior.beta2<-10
prior.gamma<-0.5

In [ ]:
# Number of trials
n<-10
# Observed x
x.obs<-3

In [ ]:
Mix.01.dat<-list(a1=prior.alpha1,b1=prior.beta1,a2=prior.alpha2,b2=prior.beta2,gamma=prior.gamma,n=n,x=x.obs)

## Model

In [ ]:
cat(
"model
    {
    x~dbin(p,n)             # Binomial likelihood. See arguments order!
    
    p<-theta[r]             # Prior is a mixture of two beta r.v 
                            # Notice the syntax: this statement says that p is
                            # *equal to* the pair of theta variates defined below

    r~dcat(g[])             # Categorical distribution:
                            # g[] is a 2-vector containing the mixing probabilities (gamma, 1-gamma)

    theta[1]~dbeta(a1,b1)   # The two beta r.v to enter the mixture
    theta[2]~dbeta(a2,b2)

    g[1]<-gamma             # Just as in p. g is equal to (gamma, 1-gamma)
    g[2]<-1-gamma
    }"
    ,file="Mix.01.jag")

## Run JAGS

In [ ]:
Mix.01.m1<-jags(data=Mix.01.dat, n.chains=3,n.iter=5000,n.burnin=500, 
        parameters.to.save=c("theta","p"), model.file="Mix.01.jag")

In [ ]:
print(Mix.01.m1)

In [ ]:
p.sample<-Mix.01.m1$BUGSoutput$sims.list$p

In [ ]:
p.sample.density<-density(p.sample)
plot(p.sample.density,lwd=2.5,col="blue",main=expression(paste("Estimated posterior pdf of ",p)),cex.main=1.6)